In [561]:
import numpy as np
import pandas as pd
from gensim.test.utils import datapath
from gensim import utils
import string
import re
from nltk.corpus import stopwords
import time

# Reading in and preprocessing scraped website text

First, separate into sentences by splitting on full stops. Then covert each line to lowercase, remove numbers, punctuation and symbols, then whitespaces like tabs, etc. Finally tokenise each sentence.

In [562]:
matthew = False
path = "C:/Users/laure/OneDrive/Documenten/Project 3.1 APG Files/"
if matthew:
    private_scrape = pd.read_excel('DATA/scrape_results/private_scrape_results_cleaned.xlsx')
    public_scrape = pd.read_excel('DATA/scrape_results/public_scrape_results_cleaned.xlsx')
else:
    private_scrape = pd.read_excel(path + "private_scrape_results_cleaned.xlsx")
    public_scrape = pd.read_excel(path + "public_scrape_results_cleaned.xlsx")

dutch_stopwords = stopwords.words('dutch')
dutch_months = ['Januari', "Februari", "Maart", "April", "Mei", "Juni", "Juli", "Augustus", "September", "Oktober", "November", "December", 'januari', "februari", "maart", "april", "mei", "juni", "juli", "augustus", "september", "oktober", "november", "december"]
words_to_remove_dutch = dutch_stopwords + dutch_months
dutch_pronouns = ['ik', 'je', 'jij', 'u', 'hij', 'ze', 'zij', 'hij', 'het', 'wij', 'we', 'jullie', 'me', 'mij', 'jou', 'hem', 'haar', 'ons', 'hen', 'hun', 'mijn', 'jouw', 'uw', 'zijn', 'ons', 'onze', 'zich', 'deze', 'die', 'dit', 'dat']
dutch_stopwords = [word for word in words_to_remove_dutch if word not in dutch_pronouns]
dutch_stopwords_and_pronouns = list(set(dutch_stopwords) | set(dutch_pronouns))

In [563]:
filter_stopwords = False
filter_pronouns = True

def preprocess_corpus(df):
    paragraphs = []
    sentences = []
    for index, row in df.iterrows():
        paragraphs.append(row.paragraph)
    
    for para in paragraphs:
        sentences.extend(re.split('[.|?|!|;|\n|\t]', para))

    for index, sentence in enumerate(sentences):
        sentence = sentence.replace('_x000D_', '')
        sentence = sentence.replace('\xa0', ' ')
        sentence = sentence.replace('–', '')
        sentence = sentence.replace('‘', '')
        sentence = sentence.replace('’', '')
        sentence = sentence.replace('”', '')
        sentence = sentence.replace('“', '')
        sentence = sentence.replace('á', 'a').replace('à', 'a').replace('ä', 'a').replace('â', 'a').replace('ã', 'a')
        sentence = sentence.replace('ó', 'o').replace('ò', 'o').replace('ö', 'o').replace('ô', 'o').replace('õ', 'o')
        sentence = sentence.replace('é', 'e').replace('è', 'e').replace('ë', 'e').replace('ê', 'e')
        sentence = sentence.replace('í', 'i').replace('ì', 'i').replace('ï', 'i').replace('î', 'i')
        sentence = sentence.replace('ú', 'u').replace('ù', 'u').replace('ü', 'u').replace('û', 'u')
        sentence = sentence.replace('ý', 'y').replace('ÿ', 'y')
        sentence = sentence.replace('ç', 'c')
        sentence = sentence.replace('ñ', 'n')
        sentence = sentence.lower()
        sentence = re.sub(r'\d+', '', sentence)
        sentence = sentence.translate(str.maketrans("","", string.punctuation))
        if filter_stopwords and filter_pronouns:
            words = sentence.split()
            words = [word for word in words if word not in dutch_stopwords_and_pronouns]
            new_sentence = " ".join(words)
        elif filter_stopwords:
            words = sentence.split()
            words = [word for word in words if word not in dutch_stopwords]
            new_sentence = " ".join(words)
        elif filter_pronouns:
            words = sentence.split()
            words = [word for word in words if word not in dutch_pronouns]
            new_sentence = " ".join(words)
        else:
            new_sentence = sentence
        if 'cms' in new_sentence:
            new_sentence = ''
        sentences[index] = new_sentence.strip()

    sentences = list(filter(None, sentences))

    return sentences


In [564]:
# Preprocess the public and private scrape results
processed_public_scrape = preprocess_corpus(public_scrape)
length = len(processed_public_scrape)
processed_private_scrape = preprocess_corpus(private_scrape)

In [565]:
# Combine public and private scrape results
processed_public_scrape.extend(processed_private_scrape)
processed_scrapes = processed_public_scrape

In [566]:
# Convert to dataframe
processed_df = pd.DataFrame(processed_scrapes, columns=['sentences'])

In [567]:
# Save to csv
filename = 'public_and_private_sentences_{}_filtered.csv'.format('all' if filter_stopwords and filter_pronouns else 'stopwords' if filter_stopwords else 'pronouns' if filter_pronouns else 'none')
if matthew:
    processed_df.to_csv('DATA/scrape_results/' + filename, index=False)

else:
    processed_df.to_csv(path + filename, index=False)